In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

from torchvision import dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
IMAGE_SIZE = 224
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
BATCH_SIZE = 16
NUM_WORKERS = 2


In [ ]:

test_transforms = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.Normalize(MEAN, STD),
    transforms.ToTensor(),
])

test_dataset_path = "../../../dataset/test"
test_dataset = datasets.ImageFolder(root=test_dataset_path, transforms=test_transforms)

test_loader = DataLoader(
    test_dataset,
    pin_memory=True,
    shuffle=False,
    num_workers=NUM_WORKERS,
    batch_size=BATCH_SIZE,
)

In [ ]:
def test_model(model, loader, criterion, device):
    model.eval()
    test_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    calc_loss = test_loss / total
    calc_acc = correct / total
    return calc_loss, calc_acc

In [ ]:
def evaluate_on_test(model, loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)

            if isinstance(outputs, tuple):
                outputs = outputs[0]

            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
    
    print(classification_report(
        all_labels, 
        all_preds,
        target_names=class_names,
        digits=2,
        zero_division=0,
        ),
    )

In [ ]:
model = models.resnet152(weights=None)
trained_model_path = "best_resnet152.pth"

model.load_state_dict(torch.load(trained_model_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()

In [ ]:
class_names = test_dataset.classes

test_loss, test_acc = test_model(model, test_loader, criterion, device)
evaluate_on_test(model, test_loader, device, class_names)